In [ ]:
#AB testi ile Bidding Dönüşümlerinin Karşılaştırılması

In [ ]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
filename="/content/ab_testing.xlsx"

In [ ]:
df_control=pd.read_excel(filename,sheet_name="Control Group")
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [ ]:
df_test=pd.read_excel(filename,sheet_name="Test Group")
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [ ]:
df_control.describe().T #maxiumum bidding yeni tanıtılanbidding yöntemi uygulanmıs grup


,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [ ]:
df_test.describe().T #average bidding eski tanıtılan uygulanmıs grup

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


In [ ]:
df_control["group"]="control"
df_test["group"]="test"

In [ ]:
df=pd.concat([df_control,df_test],axis=0,ignore_index=True)

In [ ]:
#Hipotezi tanımlama
#H0: M1=M2 control grubu ve test grubu arasında satın alım ortalamalarında ist. olarak anlamlı bi farklılık yoktur.
#H1: M1 != M2

In [ ]:
print(df_control["Purchase"].dtype)

float64


In [ ]:
print(df_control["Purchase"].unique())

[665.21125489 315.08489497 458.0837381  487.09077317 441.03404972
 519.66965589 512.92874565 747.02012325 745.98568188 470.5013668
 386.09788079 628.87236601 613.71231478 739.53718896 458.15592463
 801.79502004 400.188826   329.37703182 267.02894281 729.16154889
 666.48206712 513.57882912 487.82877428 530.67988413 585.63916672
 686.41228601 531.73272935 772.03287925 620.13587523 468.8780309
 521.25811053 729.36526198 555.00202363 619.86718802 593.02579134
 551.07240648 345.04603255 476.16812794 474.61353748 590.40601972]


In [ ]:
#H0:Normal dağılım varsayımı sağlanmaktadır.
#H1: Normal dağılım varsayımı sağlanmamaktadır.
#Sonuç:Normallik varsayımı p=0.05'ten büyük varsayım reddeedilemez.
# If the data type is numeric:
if df_control["Purchase"].dtype in ["int64", "float64"]:
    test_stat, pvalue = shapiro(df_control["Purchase"])
    print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# If the data type is categorical:
else:
    # Perform the Shapiro-Wilk test on the frequency counts of each category
    categories, counts = np.unique(df_control["Purchase"], return_counts=True)
    test_stat, pvalue = shapiro(counts)
    print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [ ]:
# If the data type is numeric:
# p value 0.05 ten büyük normallik varsayımı sağlanır.
if df_test["Purchase"].dtype in ["int64", "float64"]:
    test_stat, pvalue = shapiro(df_test["Purchase"])
    print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# If the data type is categorical:
else:
    # Perform the Shapiro-Wilk test on the frequency counts of each category
    categories, counts = np.unique(df_test["Purchase"], return_counts=True)
    test_stat, pvalue = shapiro(counts)
    print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [ ]:
#Varyansların Homojenliğinin test edilmesi
#H0: Varyanslar homojendir.
#H1: Varyanslar homojen değildir.
#Sonuç:varyansların homojenliği 0.05'ten büyük olduğu için varyansalar homojendir.
test_stat, pvalue = levene(df_control["Purchase"],df_test["Purchase"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [ ]:
#Varsayımlar sağlandığından dolayı parametrik test olan  t-testi kullanılır.
test_stat, pvalue = ttest_ind(df_control["Purchase"],df_test["Purchase"],equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


#Sonuç: p-value 0.05'ten büyük olduğu için Hipotez reddedilemez. Yani Elde edilen sonuçta Maximum Bidding yöntemi ve Average bidding yöntemi kullanılan grupların satın alımların ortalamalarında istatistiki olarak anlamlı bir fark yoktur sonucuna ulaşılabilir.  Değerlendirme yapılırken sadece satın alımlara değil,tıklanma ve kazanç gibi verilerde dikkate alınabilir.